In [11]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm

In [2]:
# 전처리 거친 데이터셋 불러오기
import pickle 
with open('train_data.pickle', 'rb') as fr:
    train_data = pickle.load(fr)
with open('test_data.pickle', 'rb') as fr:
    test_data = pickle.load(fr)

In [16]:
train = train_data['doc_cleaned'].values
test = test_data['doc_cleaned'].values

y_train = train_data['label'].values
y_test = test_data['label'].values

In [12]:
# TFIDF 주요 단어 추출 
X_train = np.array(train)
train_pos = X_train[y_train == 1]
train_neg = X_train[y_train == 0]

tfidfv = TfidfVectorizer(preprocessor=' '.join, max_features=30000).fit(train_pos)
score = {key : value for key, value in zip(sorted(tfidfv.vocabulary_, key = lambda x : tfidfv.vocabulary_[x]), tfidfv.transform(train_pos).toarray().sum(axis=0))}
top_pos = sorted(score, key = lambda x : score[x],reverse=True)[:2000]

tfidfv = TfidfVectorizer(preprocessor=' '.join, max_features=30000).fit(train_neg)
score = {key : value for key, value in zip(sorted(tfidfv.vocabulary_, key = lambda x : tfidfv.vocabulary_[x]), tfidfv.transform(train_neg).toarray().sum(axis=0))}
top_neg = sorted(score, key = lambda x : score[x],reverse=True)[:2000]

top = top_pos + top_neg

In [ ]:
# Word2Vec으로 전체 corpus 학습
from gensim.models import Word2Vec
model = Word2Vec(train, size=128, window=3, min_count=1, iter=100, sg=1)
model.wv.vectors.shape

In [13]:
# 유사 단어 단어 사전에 추가
sim = []
for pw in tqdm(top):
    sim.extend([k for k,v in model.wv.most_similar(pw)])
top.extend(sim)

100%|█████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:08<00:00, 448.99it/s]


In [14]:
# Countervectorize 
train = list(map(' '.join, train))
test = list(map(' '.join, test))
vectorizer = CountVectorizer()
vectorizer.fit(top)
X_train = vectorizer.transform(train).toarray()
X_test = vectorizer.transform(test).toarray()
print(X_train.shape, X_test.shape)

# MultinomialNB 학습
nbc = MultinomialNB()
nbc.fit(X_train, y_train)

(145791, 22514) (48995, 22514)


MultinomialNB()

In [15]:
y_pred = nbc.predict(X_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_test, y_pred)))

테스트 정확도: 0.811
